## install required packages
run the following command line in terminal

       pip install -r requirements.txt

## Import packages

In [1]:
import pandas as pd
import numpy as np
import os
import json
import obonet
import inflect
import networkx as nx
import matplotlib.pyplot as plt
import requests
import re
from openai import OpenAI
from pydantic import BaseModel
import google.generativeai as genai
from dotenv import load_dotenv
load_dotenv()


False

## Define output format

In [2]:
class cellTypeFormat(BaseModel):
       cellType: str

## Read cell clusters file 
file path set to ./Data/all.csv

In [28]:
def read_cluster(path):
      dataframe = pd.read_csv(path)
      print(dataframe['marker'])
      return dataframe 
gene_list = read_cluster('./testing/3_lung_noise.csv')
print(gene_list.iloc[0])

0       ARHGEF26,LMO7,AGER,EMP2,GENE37,RTKN2,GPRC5A,SP...
1       GENE65,MS4A1,LINC00926,BLK,SP140,CD79A,GENE48,...
2       CLDN1,GENE9,S100A2,KRT15,KRT5,FHL2,GENE32,DST,...
3       CSF3R,S100A9,S100A8,CTSS,METTL9,CD99,GENE84,VC...
4       CD2,MALAT1,GENE52,GENE30,BCL11B,CD6,CAMK4,LEF1...
                              ...                        
2515    GENE75,COX4I2,TPM2,BGN,PDGFRB,HIGD1B,PPP1R14A,...
2516    ITLN1,PRG4,HAS1,CPB1,BCHE,VIPR2,BNC1,SYT4,ROR2...
2517    AC009269.2,SCN7A,BMP5,AL135910.1,SGIP1,CARMN,C...
2518    GENE71,FCN1,RNASE2,CD300E,VCAN,FGL2,LILRB2,MPE...
2519    ASCL3,CEL,HEPACAM2,ATP6V1G3,CLCNKB,CLNK,LINC01...
Name: marker, Length: 2520, dtype: object
tissue                                                            Lung
simuround                                                            1
noise                                                             0.25
marker               ARHGEF26,LMO7,AGER,EMP2,GENE37,RTKN2,GPRC5A,SP...
manual_annotation                   

## Split genelist
split genelist to multiple chunks so that it won't exceed model input size

In [29]:
# Generate the user message for each chunk
def generate_user_message(gene_list:  pd.DataFrame, max_chunk_size=100) -> list:
    # Split the gene_list into chunks
    lung_data = gene_list[gene_list["tissue"] == "Lung"]


    return lung_data['marker'].to_list()

# Generate messages with a tunable chunk size
messages = generate_user_message(gene_list, max_chunk_size=1)  # Adjust max_chunk_size as needed
print(messages)

['ARHGEF26,LMO7,AGER,EMP2,GENE37,RTKN2,GPRC5A,SPOCK2,DST,SCEL,LAMA3,GENE83', 'GENE65,MS4A1,LINC00926,BLK,SP140,CD79A,GENE48,BANK1,CD22,VPREB3,CD79B,ADAM28', 'CLDN1,GENE9,S100A2,KRT15,KRT5,FHL2,GENE32,DST,KRT17,PERP,ITGA2,KRT19', 'CSF3R,S100A9,S100A8,CTSS,METTL9,CD99,GENE84,VCAN,S100A12,FCN1,GENE97,CD300E', 'CD2,MALAT1,GENE52,GENE30,BCL11B,CD6,CAMK4,LEF1,TRAT1,IL7R,ITK,MAL', 'GENE39,GENE7,C20orf85,DNAH6,FAM92B,RSPH1,CCDC170,CAPS,ZMYND10,CDHR3,SNTN,FANK1', 'FCGR2B,HLA-DMB,GENE63,HLA-DQA1,CLEC10A,CSF2RA,GPR183,FPR3,GENE17,HLA-DPB1,RGS1,MS4A6A', 'GENE82,C1S,LUM,ABLIM1,CFD,C3,C1R,GENE56,FBLN1,DCN,SLIT2,ADH1B', 'TFF3,TFPI,GENE30,CCL21,IGFBP7,MMRN1,PKHD1L1,PROX1,PPFIBP1,GENE28,TIMP3,TBX1', 'APOC1,C1QB,C1QA,VSIG4,SLCO2B1,GENE44,ACP5,GENE48,ITGAX,GPD1,MARCO,OLR1', 'KLRF1,GENE47,GNLY,GZMB,GENE20,FCGR3A,NKG7,PYHIN1,KLRD1,SPON2,PRF1,CD247', 'PRDX4,MANF,SSR4,XBP1,POU2AF1,TNFRSF17,GENE93,GENE24,HSP90B1,MZB1,DERL3,SEC11C', 'SFTPA2,GENE27,SLC34A2,SFTPC,LAMP3,ABCA3,SFTPA1,SFTPD,GENE23,SFTPB,NAPSA,LPCAT

## Gemeni ai

### model configuration
set the GOOGLE_API_KEY under .env file

In [5]:
genai.configure(api_key=os.getenv("GOOGLE_API_KEY"))
gemeni_model = genai.GenerativeModel("gemini-1.5-flash")
gemeni_config = genai.GenerationConfig(response_mime_type="application/json",response_schema=list[cellTypeFormat])

NameError: name 'genai' is not defined

### Send prompt to gemini

In [ ]:
earlyStop = 2
Prompt = "Identify cell types using the following tissue name and markers separately for each row. Only provide the cell type name. Do not show numbers before the name. Some can be a mixture of multiple cell types."
def annotateCell_Gemini(messages: list)->list:
    global gemeni_model, gemeni_config 
    responses = []
    i = 0
    for message in messages:
       i+=1
       if i> earlyStop:
              break
       prompt = Prompt
       for cluster in message:
              prompt += cluster
              prompt += '\n'
       print(prompt)
       response = gemeni_model.generate_content(
              prompt,
              generation_config=gemeni_config
       )
       responses.append(response.text)
    return responses


In [ ]:
annotateCell_Gemini_results = annotateCell_Gemini(messages)
print(annotateCell_Gemini_results)

Identify cell types using the following tissue name and markers separately for each row. Only provide the cell type name. Do not show numbers before the name. Some can be a mixture of multiple cell types.PBMC : MS4A1, TNFRSF13B, IGHM, IGHD, AIM2, CD79A, LINC01857, RALGPS2, BANK1, CD79B
PBMC : MS4A1, COCH, AIM2, BANK1, SSPN, CD79A, TEX9, RALGPS2, TNFRSF13C, LINC01781
PBMC : IGHM, IGHD, CD79A, IL4R, MS4A1, CXCR4, BTG1, TCL1A, CD79B, YBX3
PBMC : IGHA2, MZB1, TNFRSF17, DERL3, TXNDC5, TNFRSF13B, POU2AF1, CPNE5, HRASLS2, NT5DC2
PBMC : GZMH, CD4, FGFBP2, ITGB1, GZMA, CST7, GNLY, B2M, IL32, NKG7
PBMC : TCF7, CD4, CCR7, IL7R, FHIT, LEF1, MAL, NOSIP, LDHB, PIK3IP1
PBMC : MKI67, TOP2A, PCLAF, CENPF, TYMS, NUSAP1, ASPM, PTTG1, TPX2, RRM2
PBMC : IL7R, TMSB10, CD4, ITGB1, LTB, TRAC, AQP3, LDHB, IL32, MAL
PBMC : IL7R, CCL5, FYB1, GZMK, IL32, GZMA, KLRB1, TRAC, LTB, AQP3
PBMC : RTKN2, FOXP3, AC133644.2, CD4, IL2RA, TIGIT, CTLA4, FCRL3, LAIR2, IKZF2

Identify cell types using the following tissue name 

## GPT

### model configuration

In [ ]:
api_key = 
# import os
# os.environ["OPENAI_API_KEY"] = 
client = OpenAI(api_key = api_key)

### upload file

In [6]:
def upload_cell_cluster(filename):
       global client
       client.files.create(
              file=open(filename, "rb"),
              purpose="assistant"
       )
       print(client.files.list())

### Create Embeddings

In [31]:
# To replace the legacy openai.embeddings_utils.get_embedding function
def get_embedding(text, model="text-embedding-3-small"):
   text = str(text).replace("\n", " ")
   return client.embeddings.create(input = [text], model=model).data[0].embedding
# 加載 RAG 資料庫
cell_type_db = pd.read_csv("lung_canonical_cell_type_markers_from_literature_v2.csv")
marker_importance_db = pd.read_csv("lung_cluster_cell_type_marker_importance_v2.csv")


In [8]:
# 只要做一次就好
# 計算資料庫嵌入
cell_type_db["embedding"] = cell_type_db["markers"].apply(lambda x: get_embedding(x, model='text-embedding-ada-002'))
# marker_importance_db["embedding"] = marker_importance_db["marker"].apply(lambda x: get_embedding(x, model='text-embedding-ada-002'))

# 保存嵌入
cell_type_db["embedding"].to_csv("cell_type_embedding_v2.csv")
# marker_importance_db["embedding"].to_csv("marker_importance_embedding.csv")

KeyboardInterrupt: 

In [32]:
# 沒做上面的話，直接跑這個
import ast
cell_type_db["embedding"] = pd.read_csv("cell_type_embedding_v2.csv")["embedding"]
cell_type_db["embedding"] = cell_type_db["embedding"].apply(lambda x: np.array(ast.literal_eval(x), dtype=float))
print(cell_type_db["embedding"][0])

[-0.0022751   0.00393444  0.02531783 ... -0.02559006 -0.02050835
 -0.00551599]


In [ ]:
# [Deprecated] 這個是用來計算 marker importance 的嵌入，但是因為資料庫有更新，所以不需要這個了
import ast  # 或者用 json

if not os.path.exists("marker_importance_embedding.csv"):
    dfs = [pd.read_csv(f"splitted_data/marker_importance_embedding-{i}.csv") for i in range(1, 13)]
    marker_importance_db["embedding"] = pd.concat(dfs, ignore_index=True)["embedding"]
else:
    marker_importance_db["embedding"] = pd.read_csv("marker_importance_embedding.csv")["embedding"]
# 加載嵌入
cell_type_db["embedding"] = pd.read_csv("cell_type_embedding.csv")["embedding"]
# marker_importance_db["embedding"] = pd.read_csv("marker_importance_embedding.csv")
cell_type_db["embedding"] = cell_type_db["embedding"].apply(lambda x: np.array(ast.literal_eval(x), dtype=float))
marker_importance_db["embedding"] = marker_importance_db["embedding"].apply(lambda x: np.array(ast.literal_eval(x), dtype=float))

print(cell_type_db["embedding"][0])
print(marker_importance_db["embedding"][0])

[-0.0022751   0.00393444  0.02531783 ... -0.02559006 -0.02050835
 -0.00551599]
[ 0.00537706 -0.00582684  0.00831246 ... -0.04004051 -0.00739938
 -0.0117619 ]


### send prompt to GPT

In [35]:
from tqdm import tqdm
# Alternate the lagecy openai.embeddings_utils.search_vectors function
def cosine_similarity(a, b):
    # print(a, b, type(a), type(b), type(a[0]), type(b[0]))
    # print(np.linalg.norm(a), np.linalg.norm(b))
    return np.dot(a, b) / (np.linalg.norm(a) * np.linalg.norm(b))

def annotateCell_GPT(messages: list, n=5) -> list:
    global client
    responses = []

    for message in tqdm(messages):

        # 提取輸入的 marker
        # print(message, type(message))
        input_markers = message.split(", ")
        input_embedding = get_embedding(", ".join(input_markers))
        cell_type_db["similarity"] = cell_type_db["embedding"].apply(lambda x: cosine_similarity(x, input_embedding))
        # marker_importance_db["similarity"] = marker_importance_db["embedding"].apply(lambda x: cosine_similarity(x, input_embedding))

        # 進行檢索
        cell_type_results = (
            cell_type_db.sort_values("similarity", ascending=False)
            .head(n)
        )
        # marker_importance_results = (
        #     marker_importance_db.sort_values("similarity", ascending=False)
        #     .head(n)
        # )

        # 組合上下文
        context = "### Cell Type Database Results ###\n"
        for _,  result in cell_type_results.iterrows():
            
            context += f"Cell Type: {result['cell type']}\nMarkers: {result['markers']}\n"
            tempdb = marker_importance_db[marker_importance_db["cell_type"] == result["cell type"]]
            if not tempdb.empty:
                for marker in result['markers'].split(", "):
                    imp = tempdb[tempdb["marker"] == marker]
                    if not imp.empty:
                        context += f"Importance of {marker}: {imp['importance'].values[0]}\n"
                    else:
                        context += f"Importance of {marker}: Not Found\n"
            context += "\n"

        # 提示用戶
        message = ("### task: Identify cell types ###\n" + message + '\n'
        + "Answer is not necessary to be in the Database. "
        + "Only provide the cell type name, formatted as: {{cell_type_name}}. "
        # + "Do not show numbers before the name. "
        + "Some can be a mixture of multiple cell types. "
        # + "You cannot answer unknown. Guess if you have to. "
        # + "If you don't know the answer, output{{-1}}"
        )
        # 提供給 GPT 模型
        retry = 0
        while retry < 3:
            completion = client.chat.completions.create(
                model="gpt-4o",
                messages=[
                    {"role": "system", "content": "Identify cell types of lung tissue using the markers provided. Use the retrieved context for your decision."},
                    {"role": "user", "content": f"{context}\n\n{message}"}
                ],
                # response_format=cellTypeFormat
            )
            # print(completion.choices[0].message.parsed)
            # responses.append(completion.choices[0].message.parsed)
            print(completion.choices[0].message.content)
            candi = completion.choices[0].message.content
            candi = re.findall(r"\{\{(.*?)\}\}", candi)
            if (not candi) or "unknown" in candi[0] or "Unknown" in candi[0]:
                retry += 1
            else:
                responses.append(candi[0])
                break

        # fail to get a valid answer, try without reg
        if retry == 3:
            message = ", ".join(input_markers)
            completion = client.chat.completions.create(
                model="gpt-4o",
                messages=[
                    {"role": "system", "content": "Identify cell types of lung tissue using the markers provided. "
                    + "Only provide the cell type name, formatted as: {{cell_type_name}}. "
                    + "Do not show numbers before the name. "
                    + "Some can be a mixture of multiple cell types."
                    # + "Don't answer unknown. If must, formatted as: {{-1}}"
                    },
                    {"role": "user", "content": message}
                ],
                # response_format=cellTypeFormat
            )
            print(completion.choices[0].message.content)
            candi = completion.choices[0].message.content
            candi = re.findall(r"\{\{(.*?)\}\}", candi)
            if candi and candi[0] != "-1":
                responses.append(candi[0])
            else:
                responses.append("Unknown")
    return responses


annotateCell_GPT_results = annotateCell_GPT(messages)
print(annotateCell_GPT_results)

  0%|          | 0/2520 [00:00<?, ?it/s]

  0%|          | 1/2520 [00:00<38:50,  1.08it/s]

{{cell_type_name}}


  0%|          | 2/2520 [00:01<36:55,  1.14it/s]

{{B-cell}}


...

{{mesenchymal cells}}


100%|█████████▉| 2519/2520 [56:38<00:01,  1.59s/it]

{{monocyte}}
{{Unknown}}
{{Unknown cell type}}
{{unknown cell type}}


100%|██████████| 2520/2520 [56:41<00:00,  1.35s/it]

{{Brush cells}}
['cell_type_name', 'B-cell', 'basal epithelial cell', 'neutrophil', 'Unidentified cell type with potential lymphocyte involvement', 'pulmonary goblet cell', 'intermediate monocyte', 'pulmonary alveolar type 2 cell', 'club cell', 'macrophage', 'natural killer t cell', 'plasma cell', 'alveolar type II cells', 'mast cell', 'intermediate monocyte', 'smooth muscle cell', 'cytotoxic T cell', 'goblet cell', 'myofibroblast cell', 'natural killer t cell', 'unspecified proliferative cell type', 'pulmonary alveolar type 2 cell', 'endothelial cell', 'goblet cell', 'vascular endothelial cell', 'lung epithelial cell', 'smooth muscle cell', 'endothelial cell', 'endothelial cell', 'endothelial cell', 'goblet cell', 'goblet cell', 'megakaryocyte', 'mixture of T cells and monocytes', 'insufficient information', 'pulmonary alveolar type 2 cell', 'club cell', 'Unsupported Cell Type', 'goblet cell', 'cell_type_name: macrophage', 'natural killer cell', 'serous secreting cell', 'endothelial c

In [36]:
lung_data = gene_list[gene_list["tissue"] == "Lung"]
lung_data['RAG_cell_type'] = annotateCell_GPT_results
lung_data.to_csv("./4o_rag_GPT/[third try]3_lung_noise.csv", index=False)

## Scoring system

### result parsing

In [149]:
def json_parsing(annotateCell_results: json)->list:
       # Parse the strings into Python objects
       parsed_results = [json.loads(item.strip()) for item in annotateCell_results]
       # Flatten the results to get a list of all cell types
       cell_types = [entry['cellType'] for result in parsed_results for entry in result]
       return cell_types

In [150]:
cell_types = json_parsing(annotateCell_GPT_results)
print(cell_types)

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

### CL correspondence
match the cell_types to get CLID from Cell Ontology
search name in graph and return the corresponding clid\
if no matches found, send request to cell ontology

In [97]:
# URL for Cell Ontology (CO) OBO file
CO_URL = 'http://purl.obolibrary.org/obo/CL.obo'
OBO_FILE_PATH = 'oboNet/cl.obo'
# dictionary to store search result
NAME_TO_CLID_DICT = dict()

def load_ontology(url):
    graph = obonet.read_obo(url)
    return graph
def search_ontology(cell_name: str, ontology='cl')->str:
    global NAME_TO_CLID_DICT
    normalized_name = inflector.singular_noun(cell_name.lower()) or cell_name.lower()
    normalized_name = re.sub(r"^\(?\d+\)?\.", "", normalized_name).strip()
    # search in dict for faster access to clid and name
    if normalized_name in NAME_TO_CLID_DICT :
        return NAME_TO_CLID_DICT[normalized_name]
    
    for id, data in graph.nodes(data=True):
        if 'name' in data:
            # Singularize and normalize the graph's 'name'
            label = inflector.singular_noun(data['name'].lower()) or data['name'].lower()
            # Compare normalized names
            if label == normalized_name:
                NAME_TO_CLID_DICT[normalized_name] = (id, label)
                return id, label # Return the clid and label(name) if a match is found
    
    # If no clid name found in graph, send request to ontology api    
    # OLS API URL
    url = f"https://www.ebi.ac.uk/ols/api/search?q={normalized_name}&ontology={ontology}"

    # Make the API request
    response = requests.get(url)
    if response.status_code == 200:
        if len(response.json()['response']['docs'])==0:
          return None, None
        id = response.json()['response']['docs'][0]['obo_id']
        label = response.json()['response']['docs'][0]['label']
        NAME_TO_CLID_DICT[normalized_name] = (id, label)
        return id, label
    else:
        print(f"Error: {response.status_code}")
        return None, None

inflector = inflect.engine()
# Load the Cell Ontology
graph = load_ontology(OBO_FILE_PATH)



In [54]:
clids = []
for cell_type in cell_types:
  clids.append(search_ontology(cell_type))

print(clids)

NameError: name 'cell_types' is not defined

### Calculate distance with two CLID
#### Method 1 : shortest path length between two nodes
calculate the distance between two nodes ( manual CLID and LLM annotated CLID)

In [91]:
def calculate_difference(graph:nx.graph, clid_1:str, clid_2:str)->int:
    try:
        return nx.shortest_path_length(graph, source=clid_1, target=clid_2)
    except:
        return -1
def calculate_difference_name(graph:nx.graph, type_1:str, type_2:str)->int:
    clid_1, label_1 = search_ontology(type_1)
    clid_2, label_2 = search_ontology(type_2)
    return calculate_difference(graph, clid_1, clid_2)


In [95]:
print(calculate_difference(graph, 'CL:0002250', 'CL:0009016'))
print(calculate_difference(graph,  'CL:0009016','CL:0002250'))
# intestinal crypt stem cell 0002250
# intestinal crypt stem cell of large intestine 0009016
# should put the broader type (i.e. LLM annotated ) behind
print(calculate_difference_name(graph, 'T cells', 'Cytotoxic T cells'))
print(calculate_difference_name(graph, 'Cytotoxic T cells', 'T cells'))
print(calculate_difference_name(graph, 'T-helper cells', 'T cells'))

-1
1
{'t cell': ('CL:0000084', 't cell'), 'cytotoxic t cell': ('CL:0000910', 'cytotoxic t cell'), 't-helper cell': ('CL:0000912', 'helper T cell')}
{'t cell': ('CL:0000084', 't cell'), 'cytotoxic t cell': ('CL:0000910', 'cytotoxic t cell'), 't-helper cell': ('CL:0000912', 'helper T cell')}
-1
{'t cell': ('CL:0000084', 't cell'), 'cytotoxic t cell': ('CL:0000910', 'cytotoxic t cell'), 't-helper cell': ('CL:0000912', 'helper T cell')}
{'t cell': ('CL:0000084', 't cell'), 'cytotoxic t cell': ('CL:0000910', 'cytotoxic t cell'), 't-helper cell': ('CL:0000912', 'helper T cell')}
3
{'t cell': ('CL:0000084', 't cell'), 'cytotoxic t cell': ('CL:0000910', 'cytotoxic t cell'), 't-helper cell': ('CL:0000912', 'helper T cell')}
{'t cell': ('CL:0000084', 't cell'), 'cytotoxic t cell': ('CL:0000910', 'cytotoxic t cell'), 't-helper cell': ('CL:0000912', 'helper T cell')}
3


{'t cell': ('CL:0000084', 't cell'), 'cytotoxic t cell': ('CL:0000910', 'cytotoxic t cell'), 'helper T cell': ('CL:0000912', 'helper T cell')}
searching
{'t cell': ('CL:0000084', 't cell'), 'cytotoxic t cell': ('CL:0000910', 'cytotoxic t cell'), 'helper T cell': ('CL:0000912', 'helper T cell')}
3


#### Method 2 : calculate adjusted path-based similarity
assign score according to 'On measuring of SImilarity between tree nodes'

$$ l_a (v_i, v_j) = \frac{l(v_i, lca_{ij})+l(v_j, lca_{ij})}{1 + l(lca_{ij},t)}$$

while score equals $$ s_a(v_i, v_j) = \frac{1}{1+l_a(v_i, v_j)} = \frac{1 + l(lca_{ij},t)}{1+ l(v_i, lca_{ij})+l(v_j, lca_{ij}) + l(lca_{ij},t)}$$

In [14]:
def adjusted_path_based_score(graph, clid_1, clid_2):
       ugraph = graph.to_undirected()
       lca = nx.lowest_common_ancestor(graph, clid_1, clid_2)
       root_node = [x for x in graph.nodes() if graph.out_degree(x)>1 and graph.in_degree(x)==0]
       try:
              l_i_lca = nx.shortest_path_length(ugraph, source = clid_1, target = lca)
              l_j_lca = nx.shortest_path_length(ugraph, source = clid_2, traget = lca)
              l_lca_t = nx.shortest_path_length(ugraph, source = lca, target = root_node)
              return (1+l_lca_t) / (1+l_i_lca + l_j_lca + l_lca_t)
       except:
              return -1

In [ ]:
print(nx.is_directed_acyclic_graph(graph))
degraph= graph.copy()
nx.find_cycle(degraph, orientation='original')

while not nx.is_directed_acyclic_graph(degraph):
       trimmed_egdes = nx.find_cycle(degraph, orientation='original')
       for edge in trimmed_egdes:
              print(graph.nodes[edge[0]]['name'],edge[2],graph.nodes[edge[1]]['name'] )
              dagraph = degraph.remove_edge(edge[0], edge[1])


In [43]:
diff_result=[]
for i in range(len(clids)):
  clid_1 =clids[i] #LLM annotated CLID
  clid_2 =gene_list["manual_CLID"][i] 
  if(clid_1 != None and clid_2!="nan"):# manual annotated CLID
    difference = calculate_difference(graph, clid_2, clid_1)
    diff_result.append(difference)
  else:
    diff_result.append(-1)
  print(clid_1, clid_2)
  # print(difference)
print(diff_result)

CL:0000236 CL:0000818
CL:0000236 CL:0000787
CL:0000236 CL:0000788
CL:0000236 CL:0000980
CL:0000910 CL:0000934
CL:0000084 CL:0000895
None nan
CL:0000084 CL:0000904
CL:0000910 CL:0000905
None CL:0000815
None CL:0000900
CL:0000084 nan
CL:0000084 CL:0000907
CL:0000910 CL:0000913
CL:0000576 nan
CL:0000451 CL:0000990
CL:0000451 CL:0000990
CL:0000236 CL:0000784
CL:0000576 CL:0001054
CL:0000235 nan
[2, 3, 3, 3, -1, 3, -1, 5, -1, -1, -1, -1, 5, -1, -1, 1, 1, -1, 1, -1]


### Assign new scoring system to all.csv

In [39]:
MODELS = ['gpt4aug3', 'gpt4mar23','gpt3.5aug3', 'CellMarker2.0','SingleR', 'ScType']
MANUAL = 'manual'

In [40]:
def assign_score(gene_list: pd.DataFrame):
       global MODELS
       global MANUAL
       global graph
       gene_list_scored = pd.DataFrame()
       gene_list_scored = gene_list.copy()
       for MODEL in MODELS:
              scores = []
              for i in range(len(gene_list)):
                     clid_1 = gene_list[MANUAL+'_CLID'][i] 
                     clid_2 = gene_list[MODEL+'_CLID'][i]
                     scores.append(calculate_difference(graph, clid_1, clid_2))
              gene_list_scored[MODEL+'_aggrement_modified'] = scores
       return gene_list_scored


In [41]:
gene_list_scored = assign_score(gene_list)
print(gene_list_scored)

            dataset       tissue  \
0           Azimuth         PBMC   
1           Azimuth         PBMC   
2           Azimuth         PBMC   
3           Azimuth         PBMC   
4           Azimuth         PBMC   
...             ...          ...   
1125  tabulasapiens  Vasculature   
1126  tabulasapiens  Vasculature   
1127  tabulasapiens  Vasculature   
1128  tabulasapiens  Vasculature   
1129  tabulasapiens  Vasculature   

                                                 marker  \
0     MS4A1, TNFRSF13B, IGHM, IGHD, AIM2, CD79A, LIN...   
1     MS4A1, COCH, AIM2, BANK1, SSPN, CD79A, TEX9, R...   
2     IGHM, IGHD, CD79A, IL4R, MS4A1, CXCR4, BTG1, T...   
3     IGHA2, MZB1, TNFRSF17, DERL3, TXNDC5, TNFRSF13...   
4     GZMH, CD4, FGFBP2, ITGB1, GZMA, CST7, GNLY, B2...   
...                                                 ...   
1125  KLRD1,NKG7,XCL2,CTSW,XCL1,GNLY,GZMA,KLRB1,KLRC...   
1126  JCHAIN,MZB1,DERL3,IGHG2,IGHA2,TNFRSF17,SDC1,FC...   
1127  TFF3,PKHD1L1,PROX1,NTS,FLT4,RE

In [42]:
gene_list_scored.to_csv('Data/all_modified.csv')